# Métodos para Resolução de Sistemas Lineares #

Nesta aula, iremos implementar métodos para encontrar vetores de solução $x$ para sistemas lineares no formato $Ax=b$.

# Preliminares #

### Resolvendo sistemas triangulares ###

Normalmente, para resolver sistemas lineares, precisamos chegar de alguma forma a sistemas triangulares (inferiores ou superiores). Por que? Porque eles podem ser resolvidos pelo método das substituições retroativas.



$$ x_{i} = \frac{b_{i}-\sum\limits_{j=1}^{i-1}{a_{ij}x_{j}}}{a_{ii}} $$

e

$$ x_{i} = \frac{b_{i}-\sum\limits_{j=i+1}^{n}{a_{ij}x_{j}}}{a_{ii}} $$

Faça duas funções, que recebendo como parâmetro uma matriz $A$ triangular (superior ou inferior) e um vetor $b$, retorne o vetor solução $X$:

In [1]:
import numpy as np

In [181]:
(A[0][:0]*b[:0]).sum()

0.0

In [182]:
# Dão erro se tiver 0 na DP pq aí é indeterminado
def resolveTS(A,b): #A Triangular Superior
    x = np.zeros(len(A))
    n = len(b)
    for i in range(len(A)-1,-1,-1):
        x[i] = (b[i] - (A[i][i+1:]*x[i+1:]).sum())/A[i][i]
    return x

def resolveTI(A,b): #A Triangular Inferior
    x = np.zeros(len(b))
    for i in range(len(b)):
        x[i] = (b[i] - (A[i][:i]*x[:i]).sum())/A[i][i] # x[<intervalo_fechado>:<intervalo_aberto>]
    return x

In [169]:
A = np.array([[1,1,-2,0],[0,4,0,1],[0,0,3,7/4],[0,0,0,-5/3]])
b = [5,11,9/4,-5]
resolveTS(A,b)

array([ 1.,  2., -1.,  3.])

In [183]:
A1 = np.flip(np.flip(A,0),1)
b1 = np.flip(b,0)
resolveTI(A1,b1)

array([ 3., -1.,  2.,  1.])

### Operações $l$-elementares ###

Fazer operações $l$-elementares com matrizes numpy é bem simples: com `M[i]` você acessa a i-ésima linha:

In [89]:
import numpy as np

M = np.random.randint(-10,10,(4,4))
print(M)

print("Subtraindo uma linha por outra multiplicada por uma constante:")
M[2] = M[2] - M[1]*2
print(M)

print("Trocar duas linhas:")

M[[1,3]] = M[[3,1]]
print(M)


[[  9   5   2   9]
 [  1   5 -10  -9]
 [  3  -2  -5  -2]
 [  3   0  -7  -9]]
Subtraindo uma linha por outra multiplicada por uma constante:
[[  9   5   2   9]
 [  1   5 -10  -9]
 [  1 -12  15  16]
 [  3   0  -7  -9]]
Trocar duas linhas:
[[  9   5   2   9]
 [  3   0  -7  -9]
 [  1 -12  15  16]
 [  1   5 -10  -9]]


### Outras operações com Matrizes ###

In [6]:
print("Achar o maior elemento da matriz:")
print(M[1].max())

print("Achar o maior elemento da matriz em módulo:")
print(abs(M[:][2]).max())

# E PARA ACHAR O MAIOR ELEMENTO DE UMA LINHA?
print(M[0][:].max())

Achar o maior elemento da matriz:
6
Achar o maior elemento da matriz em módulo:
20
5


In [7]:
print("Matriz transposta:\n",M[1:3].T)

print("Determinante de M: ",np.linalg.det(M))

print("Autovalores de uma matriz:")

(a,_) = np.linalg.eig(M) 
print(a)

Matriz transposta:
 [[ -7  10]
 [  6 -12]
 [  5  20]
 [ -6  -4]]
Determinante de M:  -3350.000000000003
Autovalores de uma matriz:
[16.65465416+12.60607072j 16.65465416-12.60607072j
 -3.50192336 +0.j          2.19261504 +0.j        ]


# Métodos Exatos #

## Eliminação gaussiana ##

Para fazer a eliminação gaussiana deve-se usar os elementos da diagonal principal de A como pivos para zerar os elementos da mesma coluna.

Implemente a eliminação gaussiana simples (sem pivotação parcial):

In [146]:
def eliminacaoGaussianaSimples(A,b):
    M = np.zeros(A.shape)
    for i in range(len(A)-1):
        M[i+1:,i] = A[i+1:,i]/A[i][i]
        A[i+1:] -= M[i+1:,i].reshape((len(M[i+1:]),1))*A[i]
        b[i+1:] -= M[i+1:,i]*b[i]
    return resolveTS(A,b)

In [147]:
A = np.array([[1,1,-2,0],[-1,3,2,1],[1,2,1,2],[2,0,-2,-1]],dtype='float')
b = np.array([5,6,10,1],dtype='float')
eliminacaoGaussianaSimples(A,b)

array([ 1.,  2., -1.,  3.])

## Decomposição LU ##

A decomposição LU é mais utilizada quando a mesma matriz de coeficientes $A$ é usada para várias soluções diferentes. Por isto, ela pode ser dividida em dois passos:

- Decompor $A$ em $L$ e $U$ 
- Dados $L$, $U$ e $b$, achar a solução X

Faça as duas funções, com a decomposição ainda sem pivotação parcial:

In [158]:
def decompoeLU(A):
    L = np.zeros(A.shape)
    L[0][0] = 1
    for i in range(len(A)-1):
        L[i+1][i+1] = 1
        L[i+1:,i] = A[i+1:,i]/A[i][i]
        A[i+1:] -= L[i+1:,i].reshape((len(L[i+1:]),1))*A[i]
    return L,A # A = U

def resolveLU(L,U,b):
    y = resolveTI(L,b)
    return resolveTS(U,y)

In [185]:
A = np.array([[1,6,2,4],[3,19,4,15],[1,4,8,-12],[5,33,9,3]],dtype='float')
b = np.array([8,25,18,72],dtype='float')
L,U = decompoeLU(A)
resolveLU(L,U,b)

array([-138.,   20.,   11.,    1.])

### Pivotação Parcial ###

Sistemas onde o determinante de uma das submatrizes principais ($A_{1x1},A_{2x2},A_{3x3}...$) é igual a $0$ não podem ser resolvidos com a decomposição LU simples. Nestes casos, deve-se utilizar a pivotação parcial, onde o pivô é escolhido da linha com o maior elemento em módulo.

Contudo, é importante guardar as trocas de linha que foram efetuadas na matriz de permutações $P$. Esta matriz é uma matriz identidade com as linhas trocadas junto com a pivotação. Por exemplo, se na primeira coluna o maior elemento está na linha três, este será o primeiro pivô (a linha 1 será trocada com a 3). Neste caso, na matriz $P$ também se troca estas linhas. No fim do processo:

$PAx = Pb$

$LUx=Pb$

Então basta resolver trocando as linhas de b de através da multiplicação por P. Lembrem que multiplicar matrizes em numpy é:

`p.dot(a)` ou `dot(p,a)`

Implemente uma função que checa se a pivotação parcial é necessária e a decomposição LU com pivotação parcial:

In [207]:
def verificaPivot(A):
    for i in range(len(A)):
        if np.linalg.det(A[:i,:i]) == 0:
            return False
    return True

def LUparcial(A):
    if verificaPivot(A):
        P = np.identity(len(A))
        L[i+1][i+1] = 1
        L[i+1:,i] = A[i+1:,i]/A[i][i]
        A[i+1:] -= L[i+1:,i].reshape((len(L[i+1:]),1))*A[i]
        return L,U,P
    return L,U,P

def resolveLUpar(L,U,P,b):
    
    return x

#### Exercício ####

Rode e verifique o tempo com `%timeit` da eliminação gaussiana, decomposição LU e LU com pivotação para o sistema abaixo. No caso da LU, calcule o tempo da decomposição e da solução dos sistemas:

In [ ]:
#Sistema 1:

M1 = np.array([[1,-3,5,6], [-8,4,-1,0],[3,2,-2,7],[1,2,5,-4]])
b1 = np.array([17,29,-11,7])

#Sistema 2:

M2 = np.array([[-2,3,1,5],[5,1,-1,0],[1,6,3,-1],[4,5,2,8]])
b2 = np.array([2,-1,0,6])


## Cholesky ##

O método de Cholesky só pode ser utilizado quando a matriz for:

- Simétrica (igual a sua transposta)
- Definida positiva: 
    - Todos os elementos da diagonal principal são positivos
    - Todos os autovalores de $A$ são positivos
    - Todas as submatrizes superiores possuem determinante __positivo__.

Se for possível, o método de Cholesky é uma decomposição LU onde $U=L^{T}$, ou seja $LL^{T}x=b$.

Lembrando que na decomposição de Cholesky os elementos da diagonal principal de L são:

$$l_{jj} = \sqrt{a_{jj}-\sum\limits_{k=1}^{j-1}{l_{jk}^2}, j = 1,2,...,n}$$

E os fora da diagonal principal são:

$$ l_{ij} = \frac{a_{ij}-\sum\limits_{k=1}^{j-1}{l_{ik}l_{jk}}}{l_{jj}} $$

Desta forma, faça uma função para determinar se uma matriz pode ser resolvida via Cholesky e uma para encontrar L (para resolver pode-se usar `resolveLU(L,L.T,b)`):

In [287]:
def verificaCholesky(A):
    if (A == A.T).all() and A.all() > 0 and np.linalg.eig(A)[0].all() > 0 and verificaPivot(A):
        return True
    return False

def geraCholesky(A):
    L = np.zeros(A.shape)
    for j in range(len(A)):
        L[j][j] = np.sqrt(A[j][j] - (np.power(L[j][:j],2).sum()))
        L[j+1:,j] = (A[j+1:,j] - (L[j+1:,:j]*L[j,:j]).sum())/L[j][j]
    return L

In [289]:
A = np.array([[4,-2,2],[-2,10,-7],[2,-7,30]])
verificaCholesky(A)

True

In [276]:
geraCholesky(A)

array([[ 2.,  0.,  0.],
       [-1.,  3.,  0.],
       [ 1., -2.,  5.]])

No método de Cholesky, por ser uma matriz simétrica, podemos, ao invés de calcular o determinante normalmente, usar a seguinte definição:

$$ det(A) = det(L)det(L') $$

$$ det(A) = \bigg(\prod_{i=1}^{n}{l_{ii}}\bigg)^2 $$

In [293]:
def detCholesky(M):
    return np.power(np.prod(M.diagonal()),2)

#### Exercicios ####

1 - Verifique se os seguintes sistemas podem ser resolvidas via Choleski

2 - Compare o tempo para decomposição LU das que são possíveis com a de Choleski usando `%timeit`

3 - Compare o tempo do calculo do determinante de numpy com o determinante específico para matrizes para choleski

In [ ]:
M3 = np.array([[9,-6,3],[-6,29,-7],[3,-7,18]])
b3 = np.array([-3,-8,33])

M4 = np.array([[4,-2,4,10],[-2,2,-1,-7],[4,-1,14,11],10,-7,11,31])
b4 = np.array([2,2,-1,-2])

M5 = np.array([[16,-4,4,12],[-4,2,-1,-7],[4,-1,26,13],[12,-7,13,25]])
b5 = np.array([2,2,-1,-2])
